# Código Mestrado

Terminar tudo sem o GloVe e dps voltar pro GloVe

## Bibliotecas

In [1]:
import pandas as pd
import re
import os
import joblib
import collections
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy
import nltk
from rouge_score import rouge_scorer
from collections import Counter
import fasttext
import numpy as np
import itertools
import string
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import math

## Leitura da Base

In [2]:
if os.path.exists('lista_papers'):
    lista_papers = joblib.load('lista_papers')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/Documents_xml')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r')
        lines.append(f.readlines())
        f.close()

    lista_papers=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_papers.append(texto)

        joblib.dump(lista_papers, 'lista_papers')

In [3]:
len(lista_papers)

1009

In [4]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
papers_com_indice = dict(zip(indices, lista_papers))

In [5]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub('\/','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub('\/','', txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub("\-","",txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'^(\d)*(\.)?([0-9]{1})?', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub('\/','', txt)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub("\-","",txt)
        txt = txt.strip()
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\\t","",txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub('&amp;quot;', '', txt)
        txt = re.sub('&amp;quot;seed&amp;quot;', '', txt)
        txt = re.sub('\.+', '', txt)
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    return txt

In [6]:
def get_indices_preprocessamento(x, funcao_preprocessamento):
    """Função para obter os índices dos papers que contém a introdução, abstract ou conclusão
    obs:alterar a o parâmetro funcao_preprocessamento com:
    preprocessa_intro OU preprocessa_abstract OU preprocessa_conclusion"""
    papers_processados=[]
    indices=[]
    for i,j in zip(list(x.keys()),list(x.values())):
        try:
            papers_processados.append(funcao_preprocessamento(j))
            indices.append(i)
        except:
            None
    return indices

In [7]:
indices_papers_com_abstract   = get_indices_preprocessamento(papers_com_indice, preprocessa_abstract)
indices_papers_com_introducao = get_indices_preprocessamento(papers_com_indice, preprocessa_intro)
indices_papers_com_conclusao  = get_indices_preprocessamento(papers_com_indice, preprocessa_conclusion)

In [8]:
print('qtd de papers com abstract', len(indices_papers_com_abstract))
print('qtd de papers com introdução', len(indices_papers_com_introducao))
print('qtd de papers com conclusão', len(indices_papers_com_conclusao))

print('---------------------------------')

print('qtd total de papers', len(lista_papers))

qtd de papers com abstract 924
qtd de papers com introdução 802
qtd de papers com conclusão 831
---------------------------------
qtd total de papers 1009


In [9]:
papers_sem_abstract = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_abstract)
papers_sem_introducao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_introducao)
papers_sem_conclusao = dict((key,value) for key, value in papers_com_indice.items() if key  not in  indices_papers_com_conclusao)

Contagem dos títulos das sessões para avaliar os títulos dos papers que não possuiam `abstract` ou `introdução`ou `conclusão`

In [10]:
def contador_sessoes(x):
    """input: dicionário de papers
       output: lista contendo o título das sessões"""
    titulos_sessoes=[]
    for i in x.values():
        regIter = re.finditer('(<SECTION(.*?)>)', i, re.DOTALL|re.MULTILINE)
        textos = [ t.group(2) for t in regIter]
        titulos_sessoes.append(textos)

    x=[]
    for i in titulos_sessoes:
        for j in i:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern,str(j))
            pattern2 = r'[0-9]'
            new_string = re.sub(pattern2, '', matches[0])
            x.append(new_string.strip())

    frequency = collections.Counter(x)
    return sorted(frequency.items(), key=lambda x:x[1], reverse=True)

In [11]:
contador_sessoes(papers_sem_abstract)

[('.', 20),
 ('Introduction', 12),
 ('Acknowledgements', 6),
 ('Results', 5),
 ('', 4),
 ('Related Work', 4),
 ('Acknowledgments', 3),
 ('Abstract  Introduction', 3),
 ('Data and Evaluation', 3),
 ('Concluding Remarks', 3),
 ('Conclusion', 2),
 ('Experimental Results', 2),
 ('Conclusions and Future Work', 2),
 ('Conditional Random Fields', 2),
 ('The Joyce System in the Ulysses User Interface', 1),
 ('The Structure of Joyce', 1),
 ('The Text Planner', 1),
 ('The Sentence Planner', 1),
 ('The Linguistic Realizer', 1),
 ('An Example', 1),
 ('ENTER AGENT #&lt;information&gt; OBJECT INCOMPONENT Black Box&gt; LOCATION #&lt;PORT P&gt;',
  1),
 ('Porting the System', 1),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('Input Structure', 1),
 ('System Architecture', 1),
 ('Linguistic Knowledge Bases', 1),
 ('Interfaces', 1),
 ('System Performance', 1),
 ('Status', 1),
 ('. Corpus-based Methods', 1),
 ('Arabic Lang

contador_sessoes(papers_sem_introducao)

In [12]:
#Results, Discussion, Concluding remarks, Experiments and Results, Conclustion and Future Research
contador_sessoes(papers_sem_conclusao)

[('.', 176),
 ('Introduction', 69),
 ('Discussion', 27),
 ('', 18),
 ('Experiments', 11),
 ('Related Work', 8),
 ('Results', 8),
 ('Acknowledgment', 7),
 ('Experimental Results', 4),
 ('Evaluation.', 3),
 ('Discussion.', 3),
 ('Summary', 3),
 ('Evaluation', 3),
 ('Acknowledgement', 3),
 ('Support Vector Machines', 3),
 ('Previous Work', 3),
 ('Motivation', 3),
 ('Experiments.', 2),
 ('Summary.', 2),
 ('Related work', 2),
 ('. Results', 2),
 ('Method', 2),
 ('REFERENCES', 2),
 ('INTRODUCTION', 2),
 ('TRANSLATION DELIVERY SYSTEM', 1),
 ('TESTING AND EVALUATING MULTI-ENGINE PERFORMANCE', 1),
 ('CURRENT AND FUTURE WORK', 1),
 ('.. The Amount of Training Data Required', 1),
 ('Grammar Transformation.', 1),
 ('LM Adaptation via Sentence Retrieval.', 1),
 ('Structured Query Models.', 1),
 ('Decision Trees.', 1),
 ('Splitting of the POS Tags.', 1),
 ('Our Tagger.', 1),
 ('Motivation and Theoretical implications.', 1),
 ('A CUG Grammar  Model that Aecomodates Word', 1),
 ('Previous work.', 1),


Seleção dos papers que possuaim as 3 sessões simultâneamente

In [13]:
indice_papers=[]
for idx in list(indices_papers_com_abstract):
    if idx in list(indices_papers_com_introducao):
        if idx in list(indices_papers_com_conclusao):
            indice_papers.append(idx)

In [14]:
print('Quantidade de papers com as três sessões simulataneamente:',len(indice_papers))

Quantidade de papers com as três sessões simulataneamente: 707


In [15]:
indices_faltantes=[]
for i in indices:
    if i not in indice_papers:
        indices_faltantes.append(i)

In [16]:
print('Quantidade de papers sem as três sessões:',len(indices_faltantes))

Quantidade de papers sem as três sessões: 302


Portanto, dos 1009 papers utilizaremos 707 para o estudo do objetivo, pois somente estes possuem simultaneamente as sessões `abstract`, `introdução` e `conclusão`.

Separado 70% em treino e 30% em teste

In [17]:
indices_treino = indice_papers[0:495]
indices_teste  = indice_papers[495:708]

In [18]:
papers_de_fora = dict((key,value) for key, value in papers_com_indice.items() if key in indices_faltantes)

papers        = dict((key,value) for key, value in papers_com_indice.items() if key in indice_papers)
papers_treino = dict((key,value) for key, value in papers_com_indice.items() if key in indices_treino)
papers_teste  = dict((key,value) for key, value in papers_com_indice.items() if key in indices_teste)

Seguir o estudo com os artigos armazenados na variável `papers`. \
Os armazenados na variável `papers_de_fora` não entrarão no estudo.

# Separação das Seções

In [19]:
#Definição do Abstract
abstract=[]
for i in papers_treino.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers_treino.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers_treino.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

#Criação de arquivo para treinar o GloVe em C
adiciona = ''
for i,j,k in zip(abstract, intro, conclusion):
    adiciona = adiciona + i + ''+ j + ''+ k +' \n '
    
#open text file
text_file = open("treino_glove.txt", "w")

#write string to file
text_file.write(adiciona)

#close file
text_file.close()

# Leitura da Sumarização Padrão Ouro

In [21]:
if os.path.exists('lista_padrao_ouro'):
    lista_padrao_ouro = joblib.load('lista_padrao_ouro')
else:
    list_of_files = os.listdir('scisummnet_release1.1__20190413/top1000_complete')

    list_of_files2= []
    for i in list_of_files:
        nome = 'scisummnet_release1.1__20190413/top1000_complete/' + i + '/' + 'summary/' +os.listdir(f'scisummnet_release1.1__20190413/top1000_complete/{i}/summary')[0]
        list_of_files2.append(nome)


    lines = []
    for file in list_of_files2:
        f= open(file,'r', encoding='utf-8' )
        lines.append(f.readlines())
        f.close()

    lista_padrao_ouro=[]
    for j in lines:
        texto=''
        for i in j:
            texto = texto + str(i)
        lista_padrao_ouro.append(texto)

        joblib.dump(lista_padrao_ouro, 'lista_padrao_ouro')

In [22]:
len(lista_padrao_ouro)

1009

In [23]:
#Aplicando indices aos papers
indices           = range(0,len(lista_papers))
padrao_ouro = dict(zip(indices, lista_padrao_ouro))

In [24]:
selecao_padrao_ouro = dict((key,value) for key, value in padrao_ouro.items() if key in  indice_papers)
selecao_padrao_ouro_treino = dict((key,value) for key, value in padrao_ouro.items() if key in  indices_treino)
selecao_padrao_ouro_teste = dict((key,value) for key, value in padrao_ouro.items() if key in  indices_teste)

In [25]:
len(selecao_padrao_ouro)

707

In [26]:
len(selecao_padrao_ouro_treino)

495

# Experimento 1

## Word Embedding

#### Word 2 Vec

model_cbow = Word2Vec(tokens, min_count=1, epochs=100, sg=0)
joblib.dump(model_cbow,'model_cbow')

In [27]:
if os.path.exists('model_cbow'):
    model_cbow = joblib.load('model_cbow')
else:
    model_cbow = Word2Vec(tokens, min_count=1, epochs=100, sg=0)
    joblib.dump(model_cbow,'model_cbow')

model_sg = Word2Vec(tokens, min_count=1, epochs=100, sg=1)
joblib.dump(model_sg,'model_sg')

In [28]:
if os.path.exists('model_sg'):
    model_sg = joblib.load('model_sg')
else:
    model_sg = Word2Vec(tokens, min_count=1, epochs=100, sg=1)
    joblib.dump(model_sg,'model_sg')

In [29]:
len(model_sg.wv.key_to_index)

38854

In [30]:
len(model_cbow.wv.key_to_index)

38854

#### Fast Text

https://radimrehurek.com/gensim/models/fasttext.html

https://fasttext.cc/docs/en/python-module.html

In [31]:
s= abstract + intro + conclusion
lista_papers2=''
for i in s:
    lista_papers2= lista_papers2+ i
    
joblib.dump(lista_papers2, 'string.txt')

['string.txt']

In [32]:
model_ft = fasttext.train_unsupervised('string.txt', model='skipgram')

In [33]:
len(model_ft.get_words())

11391

In [34]:
def vetor_frases_fasttext(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_ft.get_word_vector(j)
            except:
                soma=soma+0
        x.append(soma)
    return x

## Medida de Similaridade

In [35]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*[Ii][nN][tT][rR][oO][dD][Uu][Cc][Tt][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<[Aa][Bb][sS][tT][Rr][aA][cC][tT]>)(.+?)(</[aA][bB][sS][tT][rR][aA][cC][tT]>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter1 = re.finditer('(<SECTION.*[Cc][Oo][Nn][Cc][Ll][Uu][Ss][Ii][Oo][Nn].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos1 = [ t.group(2) for t in regIter1]
    regIter2 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos2 = [ t.group(2) for t in regIter2]
    regIter3 = re.finditer('(<SECTION.*[Aa][Cc][Kk][Nn][Oo][Ww][Ll][Ee][Dd][Gg][Ee][Mm][Ee][Nn][Tt][Ss].*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos3 = [ t.group(2) for t in regIter3]
    textos = textos1
    for i in textos2:
        if i not in textos:
            textos.append(i)
    for i in textos3:
        if i not in textos:
            textos.append(i)
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        lista.append(txt)
    return txt

In [36]:
def funcao_frase_palavra(frases_abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [37]:
def vetor_frases_sg(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_sg.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

def vetor_frases_cbow(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_sg.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [38]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [39]:
#Definição do Abstract
abstract=[]
for i in papers_treino.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers_treino.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers_treino.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [40]:
def frases_palavras_vetores(secao, funcao_conversao_vetor):
    frases_intro=[]
    for i in secao:
        frases_intro.append(remove_values_from_list(i.split('.'),''))
        
    frases_intro3=[]
    for i in frases_intro:
        frases_intro2=[]
        for j in i:
            frases_intro2.append(j.strip())
        frases_intro3.append(frases_intro2)

    frases_intro = frases_intro3 
    
    
    palavras_intro=[]
    for i in frases_intro:
        palavras_intro.append(funcao_frase_palavra(i))

    vetor_intro=[]
    for i in palavras_intro:
        vetor_intro.append(funcao_conversao_vetor(i))
        
    return frases_intro,palavras_intro,vetor_intro

In [41]:
def aplicando_dicionario_similaridade(v1,v2):
    x = []
    for i,j in zip(range(len(v1)),range(len(v2))):
        x.append(dicionario_similaridade(v1[i], v2[j]))
        
    top_3_indices = []
    for i in range(len(x)):
        top_3_indices.append(sorted(x[i], key=x[i].get, reverse=True)[:3])
        
    lista_1=[]
    for i in top_3_indices:
        top3=[]
        for j in i:
            top3.append(j[0])
        lista_1.append(list(set(top3)))
        
    return x,top_3_indices,lista_1

In [42]:
def frases_interseccao(lista_1,lista_2):
    indices_objetivo=[]
    for i,j in zip(lista_1,lista_2):
        indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))
    return indices_objetivo

In [43]:
def get_frases_objetivo(indices_objetivo):
    """retorna a frase objetivo a partir dos indices"""
    frase_objetivo=[]
    for idx_f,idx_ind in zip(range(len(frases_abstract)),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(frases_abstract[idx_f][j])
            #print(idx_f)
            #print(j)
            #print(frases_abstract[idx_f][j])
        frase_objetivo.append(lista_one)
    return frase_objetivo

## Similaridade

In [44]:
def funcao_frase_palavra(frases_abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [45]:
def vetor_frases_sg(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_sg.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

def vetor_frases_cbow(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model_cbow.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [46]:
def funcao_sim_coseno(a,b):
    if ((numpy.linalg.norm(a)==0) |(numpy.linalg.norm(b)==0)):
        return 0
    else:
        valor_coseno = (numpy.dot(a, b))/(numpy.linalg.norm(a)* numpy.linalg.norm(b))
        return valor_coseno
    
def dicionario_similaridade(vetor_a,vetor_b):
    """Input: vetor correspondente às frases das seções a e b
    Output: 
    """
    dicionario={}
    for i in range(len(vetor_a)):
        for j in range(len(vetor_b)):
            dicionario[i,j]= funcao_sim_coseno(vetor_a[i],vetor_b[j])
    return dicionario

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [47]:
#Definição do Abstract
abstract=[]
for i in papers_treino.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers_treino.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers_treino.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [48]:
def frases_palavras_vetores(secao, funcao_conversao_vetor):
    """input: nome da seção (abstract, introdução, conclusão) e função que converte as frases em vetores
    output: frases, palavras e vetor correspondentes à seção de entrada"""
    frases_intro=[]
    for i in secao:
        frases_intro.append(remove_values_from_list(i.split('.'),''))
        
    frases_intro3=[]
    for i in frases_intro:
        frases_intro2=[]
        for j in i:
            frases_intro2.append(j.strip())
        frases_intro3.append(frases_intro2)

    frases_intro = frases_intro3 
    
    
    palavras_intro=[]
    for i in frases_intro:
        palavras_intro.append(funcao_frase_palavra(i))

    vetor_intro=[]
    for i in palavras_intro:
        vetor_intro.append(funcao_conversao_vetor(i))
        
    return frases_intro,palavras_intro,vetor_intro

In [49]:
def aplicando_dicionario_similaridade(v1,v2):
    """cálculo da similaridade, pegando os 3 vetores mais similares"""
    x = []
    for i,j in zip(range(len(v1)),range(len(v2))):
        x.append(dicionario_similaridade(v1[i], v2[j]))
        
    top_3_indices = []
    for i in range(len(x)):
        top_3_indices.append(sorted(x[i], key=x[i].get, reverse=True)[:3])
        
    lista_1=[]
    for i in top_3_indices:
        top3=[]
        for j in i:
            top3.append(j[0])
        lista_1.append(list(set(top3)))
        
    return x,top_3_indices,lista_1

In [50]:
def frases_interseccao(lista_1,lista_2):
    """comparando se os índices similares pertencem às duas listas (abstract e conclusão) e (abstract e intro) 
    e retorna os índices em comum"""
    indices_objetivo=[]
    for i,j in zip(lista_1,lista_2):
        indices_objetivo.append(list(set.intersection(*map(set,[i,j]))))
    return indices_objetivo

In [51]:
def get_frases_objetivo(indices_objetivo):
    """retorna a frase objetivo a partir dos indices"""
    frase_objetivo=[]
    for idx_f,idx_ind in zip(range(len(frases_abstract)),indices_objetivo):
        lista_one=[]
        for j in idx_ind:
            lista_one.append(frases_abstract[idx_f][j])
            #print(idx_f)
            #print(j)
            #print(frases_abstract[idx_f][j])
        frase_objetivo.append(lista_one)
    return frase_objetivo

In [52]:
#Word2Vec - SkipGram

#transforma em vetor
frases_intro,palavras_intro,vetor_intro                = frases_palavras_vetores(intro, vetor_frases_sg)
frases_abstract,palavras_abstract,vetor_abstract       = frases_palavras_vetores(abstract, vetor_frases_sg)
frases_conclusion,palavras_conclusion,vetor_conclusion = frases_palavras_vetores(conclusion, vetor_frases_sg)

        #Abtract e Introdução
sim_ab_intro_sg,indices_ab_intro_sg,lista_idx_ab_intro_sg = aplicando_dicionario_similaridade(vetor_abstract,vetor_intro)

        #Abtract e Conclusão
sim_ab_conc_sg,indices_ab_conc_sg,lista_idx_ab_c_sg = aplicando_dicionario_similaridade(vetor_abstract,vetor_conclusion)

#Pega os Indices em comum
indices_objetivo_sg = frases_interseccao(lista_idx_ab_intro_sg,lista_idx_ab_c_sg)

#Retorna as Frases correspondentes
frase_objetivo_sg = get_frases_objetivo(indices_objetivo_sg)

joblib.dump(frase_objetivo_sg,'frase_objetivo_sg')

['frase_objetivo_sg']

In [53]:
#Word2Vec - CBOW

#transforma em vetor
frases_intro,palavras_intro,vetor_intro_cbow                = frases_palavras_vetores(intro, vetor_frases_cbow)
frases_abstract,palavras_abstract,vetor_abstract_cbow       = frases_palavras_vetores(abstract, vetor_frases_cbow)
frases_conclusion,palavras_conclusion,vetor_conclusion_cbow = frases_palavras_vetores(conclusion, vetor_frases_cbow)

    #Abtract e Introdução
sim_ab_intro_cbow, indices_ab_intro_cbow, lista_idx_ab_intro_cbow  = aplicando_dicionario_similaridade(vetor_abstract_cbow,vetor_intro_cbow)

    #Abtract e Conclusão
sim_ab_conc_cbow, indices_ab_conc_cbow, lista_idx_ab_c_cbow = aplicando_dicionario_similaridade(vetor_abstract_cbow,vetor_conclusion_cbow)

#Pega os Indices em comum
indices_objetivo_cbow = frases_interseccao(lista_idx_ab_intro_cbow,lista_idx_ab_c_cbow)

#Retorna as Frases correspondentes
frase_objetivo_cbow = get_frases_objetivo(indices_objetivo_cbow)

joblib.dump(frase_objetivo_cbow,'frase_objetivo_cbow')

['frase_objetivo_cbow']

In [54]:
#FastText
frases_intro_ft,palavras_intro_ft,vetor_intro_ft                = frases_palavras_vetores(intro, vetor_frases_fasttext)
frases_abstract_ft,palavras_abstract_ft,vetor_abstract_ft       = frases_palavras_vetores(abstract, vetor_frases_fasttext)
frases_conclusion_ft,palavras_conclusion_ft,vetor_conclusion_ft = frases_palavras_vetores(conclusion, vetor_frases_fasttext)


#Abtract e Introdução
sim_ab_intro_ft,indices_ab_intro_ft,lista_idx_ab_intro_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_intro_ft)

#Abtract e Conclusão
sim_ab_conc_ft,indices_ab_conc_ft,lista_idx_ab_c_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_conclusion_ft)

#Indices
indices_objetivo_ft = frases_interseccao(lista_idx_ab_intro_ft,lista_idx_ab_c_ft)

#Frases
frase_objetivo_ft = get_frases_objetivo(indices_objetivo_ft)

joblib.dump(frase_objetivo_ft,'frase_objetivo_ft')

['frase_objetivo_ft']

# ROUGE

In [55]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [346]:
selecao_padrao_ouro        = list(selecao_padrao_ouro.values())
selecao_padrao_ouro_treino = list(selecao_padrao_ouro_treino.values())
selecao_padrao_ouro_teste  = list(selecao_padrao_ouro_teste.values())

AttributeError: 'list' object has no attribute 'values'

In [57]:
def pre_processamento_ouro(txt):
    txt = re.sub(r'\n',' ', txt)
    txt = re.sub(r'<.*?>','', txt)
    txt = txt.strip()
    txt = re.sub(' +', ' ', txt)
    #txt = re.sub(' . ', '.', txt).strip()
    #txt = '<sos> ' + txt.strip() + ' <eos>'
    txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
    txt = re.sub('&amp;quot;', '', txt)
    txt = re.sub('  ', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = re.sub('e.g.','',txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    txt = txt.lower()
    txt = txt.strip()
    return txt

In [58]:
def pre_processamento_ouro2(txt):
    txt = txt.strip()
    return txt

In [59]:
def calculo_rouge(frase_objetivo,lista_frases_padrao_ouro):
    rouge_total=[]
    for i,j in zip(frase_objetivo,lista_frases_padrao_ouro):
        rouge_1={}
        for h in range(len(i)):
            for z in range(len(j)):
                rouge_1[(h,z)] = (scorer.score(i[h],j[z])['rouge1'].fmeasure)
        rouge_total.append(rouge_1)
    return rouge_total

In [60]:
def get_max(dic, coord, val):
    return max(filter(lambda item: item[0][coord] == val, dic.items()), key=lambda x: x[1])

In [61]:
def max_rouge(rouge_total):
    """pega o máximo rouge score para cada frase selecionada como pertencente ao objetivo, em comparação com cada frase
    do padrão ouro"""
    maximo_rouge_por_paper =[]
    for h in rouge_total:
        x=[]
        for i in set([i[0] for i in h]):
            x.append(get_max(h,0,i))
        maximo_rouge_por_paper.append(x)
    return maximo_rouge_por_paper

In [62]:
lista_frases_padrao_ouro=[]
for i in selecao_padrao_ouro_treino:
    texto = pre_processamento_ouro(i).strip().split('.')
    texto = remove_values_from_list(texto,'')
    lista_frases_padrao_ouro.append(texto)

In [63]:
x=[]
for i in lista_frases_padrao_ouro:
    x.append(len(i))
media_ouro=np.mean(x)
media_ouro

6.864646464646465

In [64]:
rouge_ft = calculo_rouge(frase_objetivo_ft,lista_frases_padrao_ouro)
rouge_sg = calculo_rouge(frase_objetivo_sg,lista_frases_padrao_ouro)
rouge_cbow  = calculo_rouge(frase_objetivo_cbow,lista_frases_padrao_ouro)

In [65]:
rouge_cbow == rouge_sg

False

In [66]:
rouge_cbow == rouge_ft

False

In [67]:
rouge_sg == rouge_ft

False

In [68]:
maximo_rouge_por_paper_ft   = max_rouge(rouge_ft)
maximo_rouge_por_paper_sg   = max_rouge(rouge_sg)
maximo_rouge_por_paper_cbow = max_rouge(rouge_cbow)

In [69]:
def rouge_precisao_revocacao(maximo_rouge_por_paper):
    precisao=[]
    for i in maximo_rouge_por_paper:
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            precisao.append(soma/len(i))
        except:
            precisao.append(0)

    revocacao=[]
    for i,h in zip(maximo_rouge_por_paper,lista_frases_padrao_ouro):
        soma=0
        for j in i:
            soma = soma + j[1]
        #print(len(i))
        try:
            revocacao.append(soma/len(h))
        except:
            revocacao.append(0)
    return precisao, revocacao

In [70]:
precisao_sg, revocacao_sg = rouge_precisao_revocacao(maximo_rouge_por_paper_sg)

In [71]:
precisao_cbow, revocacao_cbow = rouge_precisao_revocacao(maximo_rouge_por_paper_cbow)

In [72]:
precisao_ft, revocacao_ft = rouge_precisao_revocacao(maximo_rouge_por_paper_ft)

# Experimento 1 - Conclusão

In [73]:
def analise_precisao(precisao):
    x=[]
    qtd_zero=[]
    for i in precisao:
        if i>=0.8:
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com precisao>=0.8:',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    print('media precisao:', np.mean(precisao))
    return len(x), len(qtd_zero),np.mean(precisao)

In [74]:
print(np.mean(precisao_sg))
print(np.mean(precisao_cbow))

0.7993235051807926
0.7570617628982128


In [75]:
qtd_precisao_08_cbow, qtd_zero_cbow, media_cbow = analise_precisao(precisao_cbow)

qtd de papers com precisao>=0.8: 372
% do total: 0.7515151515151515
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 85
% do total: 0.1717171717171717
media precisao: 0.7570617628982128


In [76]:
qtd_precisao_08_sg, qtd_zero_sg, media_sg = analise_precisao(precisao_sg)

qtd de papers com precisao>=0.8: 394
% do total: 0.795959595959596
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.13535353535353536
media precisao: 0.7993235051807926


In [77]:
qtd_precisao_08_ft, qtd_zero_ft,  media_ft = analise_precisao(precisao_ft)

qtd de papers com precisao>=0.8: 392
% do total: 0.7919191919191919
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 61
% do total: 0.12323232323232323
media precisao: 0.805909926956016


In [78]:
precisao_mean =  {'media_ft':media_ft,
                  'media_sg':media_sg,
                  'media_cbow':media_cbow}
max(precisao_mean, key=precisao_mean.get)

'media_ft'

In [79]:
qtd_papers_hipotese_invalida = {'qtd_zero_sg':qtd_zero_sg,
                              'qtd_zero_ft':qtd_zero_ft,
                              'qtd_zero_cbow':qtd_zero_cbow}
min(qtd_papers_hipotese_invalida, key=qtd_papers_hipotese_invalida.get)

'qtd_zero_ft'

In [80]:
qtd_precisao_alta = {'qtd_precisao_08_sg':qtd_precisao_08_sg,
                     'qtd_precisao_08_ft':qtd_precisao_08_ft,
                     'qtd_precisao_08_cbow':qtd_precisao_08_cbow}
max(qtd_precisao_alta, key=qtd_precisao_alta.get)

'qtd_precisao_08_sg'

In [81]:
def analise_revocacao(revocacao):
    x=[]
    qtd_zero=[]
    for i in revocacao:
        if i>=(1/media_ouro): #valor de 1/média qtd frases padrao ouro
            x.append(i)
        if i==0:
            qtd_zero.append(i)
    print('qtd de papers com revocação>=(1/media_ouro):',len(x))
    print('% do total:',len(x)/len(lista_frases_padrao_ouro))
    print('qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções:',len(qtd_zero))
    print('% do total:',len(qtd_zero)/len(lista_frases_padrao_ouro))
    return len(x), np.mean(revocacao)

In [82]:
print('skip-gram')
qtd_revocacao_sg, media_revocacao_sg      = analise_revocacao(revocacao_sg)

print('cbow')
qtd_revocacao_cbow,  media_revocacao_cbow   = analise_revocacao(revocacao_cbow)

print('fasttext')
qtd_revocacao_ft,  media_revocacao_ft       = analise_revocacao(revocacao_ft)

skip-gram
qtd de papers com revocação>=(1/media_ouro): 299
% do total: 0.604040404040404
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 67
% do total: 0.13535353535353536
cbow
qtd de papers com revocação>=(1/media_ouro): 291
% do total: 0.5878787878787879
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 85
% do total: 0.1717171717171717
fasttext
qtd de papers com revocação>=(1/media_ouro): 312
% do total: 0.6303030303030303
qtd de papers onde não há frases similares presentes simultaneamente nas 3 seções: 61
% do total: 0.12323232323232323


In [83]:
qtd_revocacao_alta = {'qtd_revocacao_sg':qtd_revocacao_sg,
                     'qtd_revocacao_ft':qtd_revocacao_ft,
                     'qtd_revocacao_cbow':qtd_revocacao_cbow}
max(qtd_revocacao_alta, key=qtd_revocacao_alta.get)

'qtd_revocacao_ft'

In [84]:
media_revocacao   = {'media_revocacao_sg':media_revocacao_sg,
                     'media_revocacao_ft':media_revocacao_ft,
                     'media_revocacao_cbow':media_revocacao_cbow}
max(media_revocacao, key=media_revocacao.get)

'media_revocacao_ft'

In [85]:
media_revocacao_sg

0.20777584647115963

In [86]:
media_revocacao_ft

0.21296846725291213

In [87]:
media_revocacao_cbow

0.19529128532924242

# Experimento 2

To do:
    - Pegar mais de uma frase objetivo na seleção das probabilidades;
    - Melhorar o pré-processamento de forma decente, se náo vai dar errado (remover tags, pontuação e stop words)
    - Substituir o bigrama pelo trigrama

x=1
for i in (bigrams(teste)):
    print(f'P({i[0]}|{i[1]}):')
    print(model.score(i[1], context=i[0].split()))
    a=model.score(i[1], context=i[0].split())
    x=x*a
x

N-grams are a sparse representation of language. It will give zero probability to all the words that are not present in the training corpus

from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

 Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

 Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
  Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [88]:
def preprocessa_paper(corpus: str):
    regIter = re.finditer('(<PAPER>(.*)<\/PAPER>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = txt.strip()
        txt = txt.lstrip()
        lista.append(txt)
    return txt

https://github.com/ilhamksyuriadi/Language-Modeling-Bigram/blob/master/Bigram.py

https://medium.com/analytics-vidhya/a-comprehensive-guide-to-build-your-own-language-model-in-python-5141b3917d6d

model.score('is', context='language'.split())

x=1
for i in (bigrams(teste)):
    print(f'P({i[0]}|{i[1]}):')
    print(model.score(i[1], context=i[0].split()))
    a=model.score(i[1], context=i[0].split())
    x=x*a
x

In [ ]:
paper_exemplo_treino = preprocessa_paper(papers_treino[0])

In [ ]:
len(paper_exemplo_treino.split('.'))

In [ ]:
#Lista das palavras
lista_palavras=[]
for i in paper_exemplo_treino.split('.'):
    lista_palavras.append(i.split(' '))
lista_palavras

#Bigrama de cada frase
x=[]
for frases in lista_palavras:
    x.append(bigrams(frases))

#Bigramas de cada frase
for i in x:
    for j in i:
        print(j)

In [ ]:

for i in x:
    for j in i:
        print(j)
        print(model.score(j[1], context=j[0].split()))

In [ ]:
paper_exemplo_treino = preprocessa_paper(papers_treino[0])

In [90]:
todos_papers = list(papers_treino.values())[0:5]

## Bigramas

In [142]:
def preprocessa_paper(corpus: str):
    """Pré-processa o paper.Não pode remover o ponto final"""
    regIter = re.finditer('(<PAPER>(.*)<\/PAPER>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = txt.lower()
        txt = re.sub(r'<.*?>','', txt)
        txt = re.sub(' +', ' ', txt)
        txt = re.sub("\(.*?\)","",txt)
        txt = re.sub('e.g.','',txt)
        txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
        txt = re.sub(r'\d', '', txt) #removendo demais numeros
        txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
        txt = re.sub('i\.e\.', '', txt)
        txt = re.sub('acc\.', '', txt)
        txt = re.sub('e\.g\.', '', txt)
        txt = txt.strip()
        txt = txt.lstrip()
        lista.append(txt)
    return txt

In [143]:
def preprocessa_paper2(frase: str):
    """Pré-processa as frases objetivo. Remove tudo inclusive o ponto final"""
    txt = re.sub('\\n',' ', frase)
    txt = txt.lower()
    txt = re.sub(r'<.*?>','', txt)
    txt = re.sub(' +', ' ', txt)
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub('e.g.','',txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    txt = re.sub('[!"\#\$\%\&\'()\*\+,:;<=>\?@\[\]^_`{|}\~]', '', txt)
    txt = re.sub(r'[0-9]+\.[0-9]+', '', txt)         #removendo decimal
    txt = re.sub(r'\d', '', txt) #removendo demais numeros
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt)
    txt = re.sub('<s>', '', txt)
    txt = re.sub('i\.e.', '', txt)
    txt = re.sub('acc\.', '', txt)
    txt = re.sub('e\.g\.', '', txt)
    txt = re.sub('</s>', '', txt)
    txt = txt.strip()
    txt = txt.lstrip()
    return txt

In [144]:
#Pré-Processamento das frases Objetivo
total=[]
for paper in frase_objetivo_ft:
    x=[]
    for frase in paper:
        x.append(preprocessa_paper2(frase))
    total.append(x)
frase_objetivo_ft_preprocessada = total.copy()


frase_objetivo_ft_formato_ml=[]
for i in frase_objetivo_ft_preprocessada:
    for j in i:
        frase_objetivo_ft_formato_ml.append(j.split(' '))

In [145]:
#Bigrama
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, frase_objetivo_ft_formato_ml)

model = MLE(n)
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 2979 items>


In [146]:
#Pre-processando os papers inteiros
frases_papers = []
for i in todos_papers:
    i = preprocessa_paper(i)
    frases_papers.append(i.split('.'))

#Removendo string nula e dando strip
frases_papers2=[]   
for lista in frases_papers:
    lista = [re.sub(' +', ' ',x.strip()) for x in lista if (x != '') and (x != ' ') and (x != '  ')]
    frases_papers2.append(lista)

In [147]:
#Criando uma lista de palavras, para cada frase, de cada paper
lista_palavras_papers=[]
for j in frases_papers2:
    x=[]
    for i in j:
            x.append(i.lstrip().split(' '))
    lista_palavras_papers.append(x)

In [148]:
#Criando os bigramas da lista
bigramas=[]
for papers in lista_palavras_papers:
    x=[]
    for frases in papers:
        x.append(bigrams(frases))
    bigramas.append(x)

In [149]:
lista_bigramas2=[]
for j in bigramas:
    lista_bigramas=[]  
    for i in j:
        lista_bigramas.append(list(i))
    lista_bigramas2.append(lista_bigramas)

In [150]:
# Probabilidade de cada frase do paper, com base nos bigramas
lista_paper_prob=[]
lista_paper_perp=[]
for paper in lista_bigramas2:
    lista_prob=[]
    lista_perp=[]
    for frase in paper:
        prob=1
        for bigrama in frase:
            prob = prob * (model.score(bigrama[1], context=bigrama[0].split()))
        lista_prob.append(prob)
        try:
            lista_perp.append(model.perplexity(frase))
        except:
            lista_perp.append(math.inf)
    lista_paper_prob.append(lista_prob)
    lista_paper_perp.append(lista_perp)

In [151]:
for i in lista_paper_perp:
    print(set(i))

{12.25470006987771, 31.512647310163743, inf}
{inf}
{11.311884822457355, inf}
{inf}
{inf}


In [152]:
lista_dict_probs=[]
for paper,prob in zip(frases_papers2,lista_paper_prob):
    res = {paper[i]: prob[i] for i in range(len(paper))}
    lista_dict_probs.append(res)

In [153]:
def find_key(input_dict, value):
    return {k for k, v in input_dict.items() if v == value}

In [154]:
todos_papers[0]

'<PAPER>\n  <S sid="0">TnT - A Statistical Part-Of-Speech Tagger</S>\n  <ABSTRACT>\n    <S sid="1" ssid="1">Trigrams\'n\'Tags (TnT) is an efficient statistical part-of-speech tagger.</S>\n    <S sid="2" ssid="2">Contrary to claims found elsewhere in the literature, we argue that a tagger based on Markov models performs at least as well as other current approaches, including the Maximum Entropy framework.</S>\n    <S sid="3" ssid="3">A recent comparison has even shown that TnT performs significantly better for the tested corpora.</S>\n    <S sid="4" ssid="4">We describe the basic model of TnT, the techniques used for smoothing and for handling unknown words.</S>\n    <S sid="5" ssid="5">Furthermore, we present evaluations on two corpora.</S>\n  </ABSTRACT>\n  <SECTION title="1 Introduction" number="1">\n    <S sid="6" ssid="1">A large number of current language processing systems use a part-of-speech tagger for pre-processing.</S>\n    <S sid="7" ssid="2">The tagger assigns a (unique or

In [105]:
frases_papers2[0]

['tnt - a statistical part-of-speech tagger trigramsntags is an efficient statistical part-of-speech tagger',
 'contrary to claims found elsewhere in the literature we argue that a tagger based on markov models performs at least as well as other current approaches including the maximum entropy framework',
 'a recent comparison has even shown that tnt performs significantly better for the tested corpora',
 'we describe the basic model of tnt the techniques used for smoothing and for handling unknown words',
 'furthermore we present evaluations on two corpora',
 'a large number of current language processing systems use a part-of-speech tagger for pre-processing',
 'the tagger assigns a part-ofspeech tag to each token in the input and passes its output to the next processing level usually a parser',
 'furthermore there is a large interest in part-ofspeech tagging for corpus annotation projects who create valuable linguistic resources by a combination of automatic processing and human cor

In [106]:
for i in lista_dict_probs:
    print(find_key(i,1))

{'sentences'}
{'n'}
{'and'}
set()
{'two-tailed'}


In [109]:
for i in lista_paper_prob:
    print(set(i))

{0.0, 1, 1.0574060192225445e-24, 2.243465551361857e-33}
{0.0, 1}
{0.0, 1, 5.189919387348301e-26}
{0.0}
{0.0, 1}


In [296]:
for i in lista_paper_perp:
    print(set(i))

{12.25470006987771, 31.512647310163743, inf}
{inf}
{11.311884822457355, inf}
{inf}
{inf}


In [331]:
frase_objetivo_ft[4]

['we present an unsupervised method for detecting grammatical errors by inferring negative evidence from edited textual corpora',
 'the system was developed and tested using essay-lh responses to prompts on the test of ish as a for language']

In [332]:
frases_papers2[4]

['an unsupervised method for detecting grammatical errors we present an unsupervised method for detecting grammatical errors by inferring negative evidence from edited textual corpora',
 'the system was developed and tested using essay-lh responses to prompts on the test of ish as a for language',
 'the errorrecognition system alek performs with about precision and recall',
 'a good indicator of whether a person knows the meaning of a word is the ability to use it appropriately in a sentence',
 'much information about usage can be obtained from quite a limited context choueka and lusignan found that people can typically recognize the intended sense of a polysemous word by looking at a narrow window of one or two words around it',
 'statistically-based computer programs have been able to do the same with a high level of accuracy',
 'thal of our work is to automatically identify inappropriate usage of specific vocabulary words in essays by looking at the local contextual cues around a ta

In [112]:
len(lista_paper_prob[0])

182

Note that it's very easy to make a mistake with the input to e.g. lm.fit, lm.entropy, lm.perplexity or lm.score. In fact, when writing this script, I first initially accidentally made a language model with only characters, which did lead to scores of only 0 and perplexities/entropies of infinity. Furthermore, accidentally using ("that",) rather than [("that",)] is a very easy mistake, and has big consequences:
https://github.com/nltk/nltk/issues/3065

#jeito certo de medir entropia
model.entropy([("claims",)])

#jeito certo de medir a perplexidade
text='contrary to claims'
model.perplexity(bigrams(text.split(' ')))

prod=1
for i in lista_bigramas2[0][1]:
    print(f'P({i[1]}|{i[0]}):',model.score(i[1],context=i[0].split()))
    prod = prod*model.score(i[1],context=i[0].split())
    print(model.entropy(i[1]))
    print(model.perplexity(i[1]))
print(prod)

## Trigramas

In [285]:
#Trigrama
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, frase_objetivo_ft_formato_ml)

model_tri = MLE(n)
model_tri.fit(train_data, padded_sents)
print(model_tri.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 2979 items>


In [286]:
#Criando uma lista de palavras, para cada frase, de cada paper
lista_palavras_papers=[]
for j in frases_papers2:
    x=[]
    for i in j:
            x.append(i.lstrip().split(' '))
    lista_palavras_papers.append(x)
    
#Criando os bigramas da lista
trigramas=[]
for papers in lista_palavras_papers:
    x=[]
    for frases in papers:
        x.append(trigrams(frases))
    trigramas.append(x)
    
lista_trigramas2=[]
for j in trigramas:
    lista_trigramas=[]  
    for i in j:
        lista_trigramas.append(list(i))
    lista_trigramas2.append(lista_trigramas)

In [292]:
# Probabilidade de cada frase do paper, com base nos bigramas
lista_paper_prob_tri=[]
lista_paper_perp_tri=[]
for paper in lista_trigramas2:
    lista_prob_tri=[]
    lista_perp_tri=[]
    for frase in paper:
        prob=1
        for trigrama in frase:
            prob = prob * (model_tri.score(trigrama[2], context=[trigrama[1],trigrama[0]]))
        lista_prob_tri.append(prob)
        try:
            lista_perp_tri.append(model_tri.perplexity(frase))
        except:
            lista_perp_tri.append(math.inf)
    lista_paper_prob_tri.append(lista_prob_tri)
    lista_paper_perp_tri.append(lista_perp_tri)

In [295]:
for i in lista_paper_perp_tri:
    #print(set(lista_prob_tri))
    print(set(i))

{1.9537753906637032, 1.6641416847460972, inf}
{inf}
{1.853007722166056, inf}
{inf}
{inf}


TO DO:

Aplicar o modelo de linguagem na parcela de teste tambem, para validar a perplexidade dela.




Alguns deram infinito por conta do preprocessamento: 

suspeita -> removação de hífen no preprocessamento e no outro nao
título fica junto da frase, mas não fica junto no treino do modelo

CAMINHO:
DAR O SPLIT NO </S> AO INVÉS DE DAR NO . 


DEPOIS LIMPAR

## Conclusão Experimento 2

In [319]:
lista_contador_positivo=[]
lista_contador_negativo=[]
for perplexidades_bi_paper, perplexidades_tri_paper in zip(lista_paper_perp,lista_paper_perp_tri):
    contador_positivo=0
    contador_negativo=0
    for p_bi, p_tri in zip(perplexidades_bi_paper, perplexidades_tri_paper):
        if p_tri<=p_bi:
            contador_positivo=contador_positivo+1
        else:
            contador_negativo=contador_negativo+1
        #lista_contador_negativo.append(contador_negativo)
    lista_contador_positivo.append(contador_positivo)
    lista_contador_negativo.append(contador_negativo)

In [320]:
lista_contador_negativo

[0, 0, 0, 0, 0]

In [321]:
lista_contador_positivo

[182, 159, 160, 191, 189]

Conclusão: 
    Quanto menor a perplexidade, melhor. Para todas as frases dos papers, a perplexidade do trigrama é menor ou igual à perplexidade do bigrama. Logo, o melhor modelo a ser seguido é o trigrama.

TO DO:
    Contar a qtd de frases cuja perplexidade diferente de infinito (e probabilidade diferente de 0). 
    Observar que para essa mesma quantidade, as perplexidades são menores 

In [326]:
lista_contador_menor=[]
for perplexidades_bi_paper, perplexidades_tri_paper in zip(lista_paper_perp,lista_paper_perp_tri):
    contador_menor=0
    for p_bi, p_tri in zip(perplexidades_bi_paper, perplexidades_tri_paper):
        if p_tri<p_bi:
            contador_menor=contador_menor+1
    lista_contador_menor.append(contador_menor)
lista_contador_menor

[2, 0, 1, 0, 0]

# Experimento 3

## aplicando o método da similaridade nos 30%

In [352]:
#Definição do Abstract
abstract=[]
for i in papers_teste.values():
    abstract.append(preprocessa_abstract(i))
    
#Definição da Introdução
intro=[]
for i in papers_teste.values():
    intro.append(preprocessa_intro(i))
    
#Definição da Conclusão
conclusion=[]
for i in papers_teste.values():
    conclusion.append(preprocessa_conclusion(i))
    
#Tokens
token_abstract   = []
for texto in abstract:
    token_abstract.append(texto.split())
    
token_intro   = []
for texto in intro:
    token_intro.append(texto.split())
    
token_conclusion   = []
for texto in conclusion:
    token_conclusion.append(texto.split())

tokens = token_abstract+token_intro+token_conclusion

In [354]:
#FastText
frases_intro_ft,palavras_intro_ft,vetor_intro_ft                = frases_palavras_vetores(intro, vetor_frases_fasttext)
frases_abstract_ft,palavras_abstract_ft,vetor_abstract_ft       = frases_palavras_vetores(abstract, vetor_frases_fasttext)
frases_conclusion_ft,palavras_conclusion_ft,vetor_conclusion_ft = frases_palavras_vetores(conclusion, vetor_frases_fasttext)


#Abtract e Introdução
sim_ab_intro_ft,indices_ab_intro_ft,lista_idx_ab_intro_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_intro_ft)

#Abtract e Conclusão
sim_ab_conc_ft,indices_ab_conc_ft,lista_idx_ab_c_ft = aplicando_dicionario_similaridade(vetor_abstract_ft,vetor_conclusion_ft)

#Indices
indices_objetivo_ft = frases_interseccao(lista_idx_ab_intro_ft,lista_idx_ab_c_ft)

In [355]:
indices_objetivo_ft

[[0, 1],
 [5],
 [2],
 [1, 2],
 [1],
 [2],
 [4],
 [2],
 [0, 1, 2],
 [0],
 [2, 5],
 [1],
 [0, 3],
 [0],
 [0, 5],
 [2],
 [0, 1],
 [0, 3],
 [0, 6],
 [],
 [1, 3],
 [2],
 [0, 1, 3],
 [2, 3],
 [0, 1],
 [0],
 [3, 4],
 [2],
 [1, 3],
 [0, 2],
 [0, 1],
 [0, 1, 2],
 [0, 3],
 [1],
 [0, 1],
 [1, 2],
 [1, 2],
 [2],
 [3],
 [4],
 [0],
 [1],
 [0],
 [0],
 [0, 1],
 [1, 3],
 [1],
 [1, 4],
 [0, 1],
 [0],
 [0],
 [1, 2],
 [],
 [2],
 [2],
 [3],
 [],
 [0],
 [0],
 [1, 3],
 [1, 5],
 [0, 3],
 [0],
 [0, 2],
 [0, 2],
 [0, 2],
 [1, 2],
 [1, 3],
 [0],
 [0],
 [2],
 [2, 4],
 [0],
 [0],
 [5],
 [2],
 [1],
 [0, 1],
 [],
 [0, 1],
 [0, 1],
 [0, 1, 2],
 [1, 2],
 [4],
 [0, 1, 2],
 [0, 1],
 [9, 11],
 [0],
 [1, 2],
 [2, 3],
 [2],
 [0],
 [3],
 [0, 1],
 [2, 3],
 [5],
 [0, 2],
 [0, 1, 2],
 [3],
 [0, 3],
 [3],
 [2],
 [4, 5],
 [0, 1],
 [0, 1],
 [2],
 [1],
 [],
 [4],
 [2, 3],
 [4, 6],
 [1, 6],
 [0],
 [0, 1],
 [0],
 [0, 2],
 [2],
 [1],
 [1, 3],
 [1, 2, 5],
 [0, 1],
 [0, 1],
 [5],
 [1, 3],
 [1],
 [3, 4],
 [0, 2, 4],
 [0],
 [],
 [0, 3, 4

#Frases
frase_objetivo_ft = get_frases_objetivo(indices_objetivo_ft)

#Frases
frase_objetivo_ft = get_frases_objetivo(indices_objetivo_ft)

joblib.dump(frase_objetivo_ft,'frase_objetivo_ft_teste')

frase_objetivo_ft

REVISAR A FUNCAO get_frases_objetivo pois ela nao muda o frase_abstract de acordo com o modelo [e no teste isso importa] -> mas acho q nao eh um problema

In [363]:
frase_objetivo=[]
for idx_f,idx_ind in zip(range(len(frases_abstract_ft)),indices_objetivo_ft):
    lista_one=[]
    for j in idx_ind:
        lista_one.append(frases_abstract_ft[idx_f][j])
    frase_objetivo.append(lista_one)

In [364]:
frase_objetivo

[['this paper presents an unsupervised learning algorithm for sense disambiguation that, when trained on unannotated ish text, rivals the performance of supervised techniques that require time-consuming hand annotations',
  'the algorithm is based on two powerful constraints &#; that words tend to have one sense per discourse and one sense per collocation &#; exploited in an iterative bootstrapping procedure'],
 ['we also discuss how the hybrid generation model can be used to simplify current generators and enhance their portability, even when perfect knowl is in principle obtainable'],
 ['this work is based on the following premises:  grammars are too complex and detailed to develop manually for most interesting domains;  parsing models must rely heavily on lexical and contextual information to analyze sentences accurately; and  existing n-grain modeling techniques are inadequate for parsing models'],
 ['this study suggests that the identification of word translations should also be p

In [366]:
papers_teste[746]

'<PAPER>\n  <S sid="0">Unsupervised Word Sense Disambiguation Rivaling Supervised Methods</S>\n  <ABSTRACT>\n    <S sid="1" ssid="1">This paper presents an unsupervised learning algorithm for sense disambiguation that, when trained on unannotated English text, rivals the performance of supervised techniques that require time-consuming hand annotations.</S>\n    <S sid="2" ssid="2">The algorithm is based on two powerful constraints &#8212; that words tend to have one sense per discourse and one sense per collocation &#8212; exploited in an iterative bootstrapping procedure.</S>\n    <S sid="3" ssid="3">Tested accuracy exceeds 96%.</S>\n  </ABSTRACT>\n  <SECTION title="1 Introduction" number="1">\n    <S sid="4" ssid="1">This paper presents an unsupervised algorithm that can accurately disambiguate word senses in a large, completely untagged corpus.1 The algorithm avoids the need for costly hand-tagged training data by exploiting two powerful properties of human language: Moreover, langu

### ROUGE-N

## Aplicando o modelo de linguagem